In [1]:
!pip3 list

Package                      Version
---------------------------- -----------
absl-py                      1.3.0
astunparse                   1.6.3
autocommand                  2.2.2
backports.csv                1.0.7
beautifulsoup4               4.11.1
blis                         0.7.9
bs4                          0.0.1
cachetools                   5.2.0
catalogue                    2.0.8
certifi                      2022.12.7
cffi                         1.15.1
charset-normalizer           2.1.1
cheroot                      9.0.0
CherryPy                     18.8.0
click                        8.1.3
colorama                     0.4.6
confection                   0.0.3
contourpy                    1.0.6
cryptography                 39.0.0
cycler                       0.11.0
cymem                        2.0.7
Cython                       0.29.32
docx                         0.2.4
en-core-web-sm               3.4.1
feedparser                   6.0.10
flatbuffers                  22.12.

In [2]:
import sys
# !{sys.executable} -m pip install tensorflow-addons[tensorflow]

# import tensorflow
# print(tensorflow.__version__)

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

kaggle/input\cord-ner-full.parquet.gzip
kaggle/input\covid_subset_3perc.csv
kaggle/input\ner_dataset.csv
kaggle/input\entity-annotated-corpus\ner.csv


In [19]:
import pandas as pd
#df = pd.read_csv("kaggle/input/entity-annotated-corpus/ner.csv", encoding = "ISO-8859-1", on_bad_lines="skip")
data = pd.read_csv("kaggle/input/covid_subset_3perc.csv", encoding="latin1")

# data = data.loc[:,["word","entity"]]
data.head(10)

,word,entity,sentence
0,angiotensin-converting,B-GENE_OR_GENOME,0
1,enzyme,I-GENE_OR_GENOME,0
2,2,I-GENE_OR_GENOME,0
3,ace2,B-GENE_OR_GENOME,0
4,as,Other,0
5,a,Other,0
6,sars-cov-2,B-CORONAVIRUS,0
7,receptor,B-CHEMICAL,0
8,molecular,Other,0
9,mechanisms,Other,0


In [20]:
data.groupby(['sentence'], sort=False).size().max()

512

In [4]:
data[data['entity'] == 'B-FOOD'].value_counts()

word     entity  sentence
chips    B-FOOD  2751        7
eggs     B-FOOD  3509        6
                 3505        6
chips    B-FOOD  2754        5
                 2531        4
                            ..
eggs     B-FOOD  3511        1
ensures  B-FOOD  242         1
                 571         1
                 607         1
vodka    B-FOOD  3426        1
Length: 259, dtype: int64

In [5]:
class SentenceGetter(object):
    
    def __init__(self, dataset):
        self.n_sent = 1
        self.dataset = dataset
        self.empty = False
        agg_func = lambda s: [(w, t) for w,t in zip(s["word"].values.tolist(),
                                                        s["entity"].values.tolist())]
        self.grouped = self.dataset.groupby("sentence").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None
          
getter = SentenceGetter(data)
sentences = getter.sentences

In [6]:
print(sentences[0])

[('angiotensin-converting', 'B-GENE_OR_GENOME'), ('enzyme', 'I-GENE_OR_GENOME'), ('2', 'I-GENE_OR_GENOME'), ('ace2', 'B-GENE_OR_GENOME'), ('as', 'Other'), ('a', 'Other'), ('sars-cov-2', 'B-CORONAVIRUS'), ('receptor', 'B-CHEMICAL'), ('molecular', 'Other'), ('mechanisms', 'Other'), ('and', 'Other'), ('potential', 'Other'), ('therapeutic', 'Other'), ('target', 'Other'), ('sars-cov-2', 'B-CORONAVIRUS'), ('has', 'Other'), ('been', 'Other'), ('sequenced', 'Other'), ('[3]', 'Other'), ('.', 'Other'), ('a', 'Other'), ('phylogenetic', 'B-EVOLUTION'), ('analysis', 'Other'), ('[3,', 'Other'), ('4]', 'Other'), ('found', 'Other'), ('a', 'Other'), ('bat', 'B-WILDLIFE'), ('origin', 'Other'), ('for', 'Other'), ('the', 'Other'), ('sars-cov-2', 'B-CORONAVIRUS'), ('there', 'Other'), ('is', 'Other'), ('a', 'Other'), ('diversity', 'Other'), ('of', 'Other'), ('possible', 'Other'), ('intermediate', 'B-NORP'), ('hosts', 'I-NORP'), ('for', 'Other'), ('sars-cov-2', 'B-CORONAVIRUS'), ('including', 'Other'), ('pan

In [14]:
from math import nan

words = list(set(data["word"].values))
n_words = len(words)

tags = []
for tag in set(data["entity"].values):
    if tag is nan or isinstance(tag, float):
        tags.append('Other')
    else:
        tags.append(tag)
n_tags = len(tags)


In [15]:
word2idx = {w: i for i, w in enumerate(words)}
print(len(list(word2idx.items())), list(word2idx.items())[:10])
tag2idx = {t: i for i, t in enumerate(tags)}
print(len(list(tag2idx.items())), list(tag2idx.items())[:10])
idx2tag = {v: k for k, v in tag2idx.items()}
print(len(list(idx2tag.items())), list(idx2tag.items()))

71325 [('execution,', 0), ('1)-the', 1), ('robust,', 2), ('pfastbac', 3), ('project.conclusionthe', 4), ('jinri', 5), ('reductions', 6), ('poria', 7), ('cantly', 8), ('market)', 9)]
123 [('I-MONEY', 0), ('I-VIRUS', 1), ('B-LANGUAGE', 2), ('I-CHEMICAL', 3), ('I-EDUCATIONAL_ACTIVITY', 4), ('I-WORK_OF_ART', 5), ('I-FOOD', 6), ('I-LAW', 7), ('B-MACHINE_ACTIVITY', 8), ('I-CORONAVIRUS', 9)]
123 [(0, 'I-MONEY'), (1, 'I-VIRUS'), (2, 'B-LANGUAGE'), (3, 'I-CHEMICAL'), (4, 'I-EDUCATIONAL_ACTIVITY'), (5, 'I-WORK_OF_ART'), (6, 'I-FOOD'), (7, 'I-LAW'), (8, 'B-MACHINE_ACTIVITY'), (9, 'I-CORONAVIRUS'), (10, 'I-QUANTITY'), (11, 'I-FAC'), (12, 'B-MATERIAL'), (13, 'B-GPE'), (14, 'I-GROUP'), (15, 'I-EUKARYOTE'), (16, 'I-ORG'), (17, 'B-LAW'), (18, 'B-DAILY_OR_RECREATIONAL_ACTIVITY'), (19, 'B-ANATOMICAL_STRUCTURE'), (20, 'B-EDUCATIONAL_ACTIVITY'), (21, 'I-GENE_OR_GENOME'), (22, 'B-EVENT'), (23, 'I-LABORATORY_OR_TEST_RESULT'), (24, 'I-PHYSICAL_SCIENCE'), (25, 'I-LOC'), (26, 'B-DISEASE_OR_SYNDROME'), (27, 'B-

In [17]:
from keras_preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

maxlen = max([len(s) for s in sentences])

X = [[word2idx[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=maxlen, sequences=X, padding="post",value=n_words - 1)

y = [[tag2idx[w[1]] for w in s] for s in sentences]
y = pad_sequences(maxlen=maxlen, sequences=y, padding="post", value=tag2idx["Other"])
y = [to_categorical(i, num_classes=n_tags) for i in y]

# Split train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
np.set_printoptions(threshold=sys.maxsize)
print(X[0])

print(np.round(y[0], 5))


[47332  8973  6920 65212 25489 54274 53280 69807 24345 33273   464 67492
  2875 15932 53280 45674 67812  4918 22086 64861 54274 36999 44429 68611
 45815 58908 54274 71057  5582 12087 10000 53280 12458 30806 54274 18576
 35776 16324 64664  2995 12087 53280 37544 25557 53392  1437  3784   464
 67557 41478 64861 12458 66542 47429 47381 35776 53280 64518 10000 62616
 50881 55054 34336 11916  2996 65798  1335  7116 33031 58908 29683 10000
 41088 12398 35776 53280   464 50881 55054 28526 36246 49462 35401 18142
 25906 10000  6629 25546 29683 12286 10473 19932 43134 52775 21021 41088
 16885 45674 54274 10953 67273  1243 67744 71200 65212 50917 19710  2190
 23399 66421   464 41625  9586 44429 45414 64861 53280   464 50881 55054
 41088 12398 24539 61079  4660 13060 25906 66770 69785 29821 33031 56202
 10687 10000 53280   464 50881 55054 41088 12398 28526 54274 28629  4277
 35776 63509 54577 31285 64861  2154  1335  7116 36029 10837 29683 47882
 35692 43294 25906 10000 53280  6629 25546 29429 48

In [41]:
# !pip install tensorflow-addons[tensorflow]
#!{sys.executable} -m pip install git+https://www.github.com/keras-team/keras-contrib.git

In [2]:
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
import keras as k
import tensorflow as tf
print(tf.__version__)
import tensorflow_addons as tfa
print(tfa.__version__)
#from keras_contrib.layers import CRF
from tensorflow.keras import Model, Input

print(tf.config.list_physical_devices('GPU'))
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
 

input = Input(shape=(140,))
word_embedding_size = 150

# Embedding Layer
model = Embedding(input_dim=n_words, output_dim=word_embedding_size, input_length=140)(input)

# BI-LSTM Layer
model = Bidirectional(LSTM(units=word_embedding_size, 
                           return_sequences=True, 
                           dropout=0.5, 
                           recurrent_dropout=0.5, 
                           kernel_initializer=k.initializers.he_normal()))(model)
model = LSTM(units=word_embedding_size * 2, 
             return_sequences=True, 
             dropout=0.5, 
             recurrent_dropout=0.5, 
             kernel_initializer=k.initializers.he_normal())(model)

# TimeDistributed Layer
model = TimeDistributed(Dense(n_tags, activation="relu"))(model)  

# CRF Layer
crf = tfa.layers.CRF(n_tags)

out = crf(model)  # output
model = Model(input, model)

2.10.1
0.19.0
[]
Num GPUs Available:  0


NameError: name 'n_words' is not defined

In [55]:
#!{sys.executable} -m pip install tensorflow-gpu
#!{sys.executable} -m pip list

Package                      Version
---------------------------- -----------
absl-py                      1.2.0
argon2-cffi                  21.3.0
argon2-cffi-bindings         21.2.0
asttokens                    2.0.8
astunparse                   1.6.3
attrs                        22.1.0
backcall                     0.2.0
beautifulsoup4               4.11.1
bleach                       5.0.1
blis                         0.7.8
cachetools                   5.2.0
catalogue                    2.0.8
certifi                      2022.6.15
cffi                         1.15.1
charset-normalizer           2.1.1
click                        8.1.3
colorama                     0.4.5
contourpy                    1.0.6
cycler                       0.11.0
cymem                        2.0.6
debugpy                      1.6.3
decorator                    5.1.1
defusedxml                   0.7.1
dominate                     2.7.0
en-core-web-sm               3.4.0
entrypoints                  0.4
exec


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: C:\Users\kreti\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [52]:
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

#Optimiser 
adam = k.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999)

# Compile model
#model.compile(optimizer=adam, loss=tf.keras.losses.Loss(), metrics=tf.keras.metrics.Accuracy())

model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer='adam', metrics=['accuracy'])

#model.compile(optimizer=adam, loss=crf.loss_function, metrics=[crf.accuracy, 'accuracy'])

model.summary()

# Saving the best model only
filepath="ner-bi-lstm-td-model-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

print(X_train)
print(np.array(y_train).astype(int))

# Fit the best model
history = model.fit(X_train, np.array(y_train), batch_size=256, epochs=2, validation_split=0.1, verbose=1, callbacks=callbacks_list)



Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 140)]             0         
                                                                 
 embedding_3 (Embedding)     (None, 140, 150)          4525950   
                                                                 
 bidirectional_3 (Bidirectio  (None, 140, 300)         361200    
 nal)                                                            
                                                                 
 lstm_7 (LSTM)               (None, 140, 300)          721200    
                                                                 
 time_distributed_3 (TimeDis  (None, 140, 18)          5418      
 tributed)                                                       
                                                                 
Total params: 5,613,768
Trainable params: 5,613,768
Non-tra

In [4]:
# Plot the graph 
plt.style.use('ggplot')

def plot_history(history):
    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(accuracy) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, accuracy, 'b', label='Training acc')
    plt.plot(x, val_accuracy, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

plot_history(history)

NameError: name 'plt' is not defined

In [3]:
def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i])
        out.append(out_i)
    return out
test_pred = model.predict(X_test, verbose=1)   
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_test)

NameError: name 'model' is not defined

In [ ]:
#! pip install seqeval

In [ ]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
print("F1-score: {:.1%}".format(f1_score(test_labels, pred_labels)))


In [ ]:
# ! pip install sklearn_crfsuite

In [ ]:
from  sklearn_crfsuite.metrics import flat_classification_report  
report = flat_classification_report(y_pred=pred_labels, y_true=test_labels)
print(report)


In [ ]:
TP = {}
TN = {}
FP = {}
FN = {}
for tag in tag2idx.keys():
    TP[tag] = 0
    TN[tag] = 0    
    FP[tag] = 0    
    FN[tag] = 0    

def accumulate_score_by_tag(gt, pred):
    """
    For each tag keep stats
    """
    if gt == pred:
        TP[gt] += 1
    elif gt != 'O' and pred == 'O':
        FN[gt] +=1
    elif gt == 'O' and pred != 'O':
        FP[gt] += 1
    else:
        TN[gt] += 1

In [ ]:
for i, sentence in enumerate(X_test):
    y_hat = np.argmax(test_pred[0], axis=-1)
    gt = np.argmax(y_test[0], axis=-1)
    for idx, (w,pred) in enumerate(zip(sentence,y_hat)):
        accumulate_score_by_tag(idx2tag[gt[idx]],tags[pred])

In [ ]:
for tag in tag2idx.keys():
    print(f'tag:{tag}')    
    print('\t TN:{:10}\tFP:{:10}'.format(TN[tag],FP[tag]))
    print('\t FN:{:10}\tTP:{:10}'.format(FN[tag],TP[tag]))    